In [36]:
from nltk.corpus import stopwords
from nltk import *
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [37]:
my_file = pd.read_csv("/home/bjit/testing_work/final__csv.csv")
my_file = my_file.sample(frac=1)
print(my_file.head())

                            URL                 Family  Class
27405              nxakdrp.info     shifu_dga_6ae3f080      1
53756              google.co.mz                   Good      0
9196        uussxbwdldldoxho.eu    emotet_dga_3f6db070      1
45489             valuewalk.com                   Good      0
6561   zjnzfctktonlspuanzlr.com  dircrypt_dga_bd00c68c      1


In [76]:
my_file['URL'] = my_file['URL'].apply(lambda x: re.sub('[!@#$:).;,?&]', " ", x.lower()))

In [77]:
print(my_file.head())

                            URL                 Family  Class
27405              nxakdrp info     shifu_dga_6ae3f080      1
53756              google co mz                   Good      0
9196        uussxbwdldldoxho eu    emotet_dga_3f6db070      1
45489             valuewalk com                   Good      0
6561   zjnzfctktonlspuanzlr com  dircrypt_dga_bd00c68c      1


In [78]:
To_Process=my_file[['URL', 'Class']]

In [79]:
train, test = train_test_split(To_Process, test_size=0.2)

In [80]:
MAX_SEQUENCE_LENGTH = 300
MAX_NB_WORDS = 20000

In [81]:
from keras.preprocessing.text import Tokenizer

In [82]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train.URL)
train_sequences = tokenizer.texts_to_sequences(train.URL)
test_sequences = tokenizer.texts_to_sequences(test.URL)

In [83]:
word_index = tokenizer.word_index
# print(tokenizer.word_index)
# total words in the corpus
print('Found %s unique tokens.' % len(word_index))

Found 55511 unique tokens.


In [84]:
# get only the top frequent words on train
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
# get only the top frequent words on test
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print(train_data.shape)
print(test_data.shape)

(63564, 300)
(15892, 300)


In [85]:
train_labels = train['Class']
test_labels = test['Class']

In [60]:
train_labels.shape
test_labels.shape

(15892,)

In [48]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)

In [49]:
print(le.classes_)
print(np.unique(train_labels, return_counts=True))
print(np.unique(test_labels, return_counts=True))

[0 1]
(array([0, 1]), array([32082, 31482]))
(array([0, 1]), array([7917, 7975]))


In [52]:
labels_train = (np.asarray(train_labels))
labels_test = (np.asarray(test_labels))
print('Shape of data tensor:', train_data.shape)
print('Shape of label tensor:', labels_train.shape)
print('Shape of label tensor:', labels_test.shape)

Shape of data tensor: (63564, 300)
Shape of label tensor: (63564,)
Shape of label tensor: (15892,)


In [53]:
EMBEDDING_DIM = 100
print(MAX_SEQUENCE_LENGTH)

300


In [54]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D, Conv1D, SimpleRNN
from keras.models import Model
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential

In [62]:
print('Training CNN 1D model.')
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.5))
model.add(Conv1D(128, 5, activation="relu"))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Conv1D(128, 5, activation="relu"))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(2, activation="softmax"))
model.compile(loss='binary_crossentropy', optimizer="rmsprop", metrics=['acc'])

Training CNN 1D model.


In [86]:
model.fit(train_data, labels_train,
 batch_size=256,
 epochs=5,
 validation_data=(test_data, labels_test))

Epoch 1/5
249/249 [==============================] - 49s 199ms/step - loss: 0.6931 - acc: 0.5015 - val_loss: 0.6931 - val_acc: 0.4982
Epoch 2/5
249/249 [==============================] - 50s 202ms/step - loss: 0.6931 - acc: 0.5004 - val_loss: 0.6931 - val_acc: 0.5018
Epoch 3/5
249/249 [==============================] - 50s 202ms/step - loss: 0.6931 - acc: 0.4998 - val_loss: 0.6931 - val_acc: 0.4982
Epoch 4/5
249/249 [==============================] - 46s 184ms/step - loss: 0.6931 - acc: 0.5012 - val_loss: 0.6931 - val_acc: 0.5018
Epoch 5/5
249/249 [==============================] - 46s 184ms/step - loss: 0.6931 - acc: 0.5030 - val_loss: 0.6931 - val_acc: 0.4982


In [64]:
predicted=model.predict(test_data)
print(predicted)

[[0.49974963 0.50025034]
 [0.49974963 0.50025034]
 [0.49974963 0.50025034]
 ...
 [0.49974963 0.50025034]
 [0.49974963 0.50025034]
 [0.49974963 0.50025034]]


In [73]:
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score

TypeError: precision_recall_fscore_support() got an unexpected keyword argument 'normalize'